## 第一个爬虫的开发

In [ ]:
from urllib.request import urlopen
#引进打开浏览器的模块

url="http://www.baidu.com"

resp=urlopen(url)
#print(resp.read())输出的是字节，需要手动decode，找charset（基本输出的结果中就有，一般为utf-8/gbk）

print(resp.read().decode("utf-8"))

实际上，在浏览器中打开百度页面，首先百度服务器返回的是页面源代码（html,css,js）
通过decode返回后显示的也是页面源代码

In [ ]:
with open("mybaidu.html", mode="w", encoding="utf-8") as f:
    f.write(resp.read().decode("utf-8"))

## 在jupyter中实现load

### load网页源代码

在jupyter nb中，有更简单的方法看到网页源代码  
%load URL
用这种方法加载出来的源代码与开发者工具里elements页面下的代码一样，都是包含脚本（script）的内容，要注意可能不包含想要的数据。

In [ ]:
#忘记了可以自行重新尝试，但是最好不要run了，因为会让页面很长
%load http://www.baidu.com

### load本地Python文件

另一种加载本地Python文件的方法也类似  
%load Python文件的绝对路径  
在加载这个Python文件后，这行代码本身会被注释掉，再次run this cell会run整个py文件的code

如果不希望load只想run：  

In [ ]:
%run Python文件的绝对路径  
!python3 Python文件的绝对路径  
!python3 Python文件的绝对路径

### 获取当前位置的绝对路径

第一种方法是直接通过代码：

In [1]:
%pwd

'C:\\Users\\18810'

第二种方法是!shell命令  
***这部分还没查***

In [2]:
!cd
#windows 不用!pwd

C:\Users\18810


In [3]:
!dir
#windows 不用!ls

 驱动器 C 中的卷是 Windows-SSD
 卷的序列号是 4CD7-65AC

 C:\Users\18810 的目录

2021/12/11  22:30    <DIR>          .
2021/12/11  22:30    <DIR>          ..
2021/12/07  22:22    <DIR>          .conda
2021/12/07  21:34                60 .condarc
2021/08/28  14:27    <DIR>          .idlerc
2021/12/11  22:23    <DIR>          .ipynb_checkpoints
2020/08/13  22:32    <DIR>          .ipython
2021/12/08  12:03    <DIR>          .jupyter
2020/08/13  22:33    <DIR>          .matplotlib
2021/07/03  00:02    <DIR>          .Origin
2021/07/03  00:02    <DIR>          .QtWebEngineProcess
2020/08/08  23:08             1,024 .rnd
2020/08/14  01:19    <DIR>          .spyder-py3
2020/04/27  22:23    <DIR>          .texlive2020
2021/03/19  23:07    <DIR>          3D Objects
2020/10/02  21:47    <DIR>          ado
2021/12/03  23:05           338,935 API的使用.html
2021/03/19  23:07    <DIR>          Contacts
2020/05/30  23:17    <DIR>          Creative Cloud Files
2021/11/05  09:46    <DIR>          Desktop
2021/12/01  10:

In [ ]:
!pip install matplotlib

# 浏览器

## 网络请求

本地向服务器提交包含标签的请求，服务器拼装HTML，返回包含HTML的响应内容，在本地电脑的浏览器上执行HTML  

### 两种网站  
#### HTML里包含数据内容  
#### HTML里不包含数据内容   
本地电脑发送请求后，服务器返回的响应中只有页面结构html而不包含数据；  
本地电脑收到页面结构后先展示，在执行的页面中隐含了一些脚本  
本地电脑产生新的请求，新的url，此次服务器返回的响应中只有数据；  
最终在本地电脑上把数据和结构结合在一起呈现给用户。  
原因：将数据和页面结构分开管理-分布式，多台服务器减小压力  


**e.g.:豆瓣**  
豆瓣的网页源代码只有HTML结构没有数据  
F12-network，preserve log前打钩  
clear后重新访问，通过XHR只看数据，在reponse里可以看到响应，preview和headers里可以看到数据


**F12界面的使用**  
主要关注elements console source network这四个板块  
*elements*：鼠标在页面滑动可显示相应代码区域，如果删除某部分代码，网页中的相应内容也会实时变动删除。可作参考，但可能与页面源代码不同，script脚本会加工一些内容，加工后的数据可能是错误的  
*console*：控制台能写代码，gs调试（逆向gs）  
*source*：所有资源整合处，包括页面源代码，脚本，图片  
*network*：抓包工具，所有网络请求和返回的响应都会在其中体现。XHR可以筛选网络请求，使其只剩加在数据的网络请求（preview），preserve log能保留之前的网络请求的内容。

## http协议

常见协议：tcp/ip,soap,http,smtp  
http：heper text transfer protocol(超文本传输)用于www服务器传输超文本到本地浏览器的协议。  

http协议把一条消息分为三大块内容（无论响应还是请求）  
***请求***  
请求行：**请求方式**（get/post），请求URL地址，协议  
请求头：服务器要使用的附加信息（**cookie，token**）  
请求体：请求参数  
***响应***  
状态行：协议，状态码（处理请求的情况，200-ok,302-重定向，404-错误URL，500-超出服务器能力错误）  
响应头：客户端要使用的一些附加信息（cookie，验证信息，解密key，token）  
响应体：服务器返回的真正客户端要使用的内容（HTML，json）  


### 请求头中常见的重要内容

**user-agent**：请求载体的身份标识  
**referer**：防盗链  
**cookie**：本地字符串数据信息（用户登录信息，反爬的token）

### 响应头中一些重要内容  
**cookie**：本地字符串数据信息（用户登录信息，反爬的token）  
**其他token**：一些莫名其妙的字符，用于防止攻击和反爬  


### 请求方式  
**get**：显示提交  
**post**：隐示提交

# requests模块入门
前面用的是URLlib抓取页面源代码，这是Python内置的一个模块但不是常用的爬虫工具。  
常用的抓取页面的模块是一个第三方模块requests，它的优点是比URLlib简单，处理各种请求都比较方便。
因为是第三方，所以需要安装。   
## 安装模块  
### 直接从国外原网站安装

In [ ]:
!pip install requests

### 通过清华的镜像网站

In [ ]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple some-package

也可以直接将清华的cran设为默认

In [ ]:
!pip install pip -U
!pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple

如果pip默认源的网络连接较差，临时使用以下镜像网站来升级pip

In [ ]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple pip -U

## tests：网页源代码

In [1]:
import requests

In [2]:
#爬取百度页面源代码
url="http://www.baidu.com"
resp=requests.get(url)

resp.encoding="utf-8"
#不要忘记对resp解码

In [3]:
print(resp)
#显示响应状态

<Response [200]>


In [4]:
print(resp.text)
#显示响应文本信息：页面源代码

<!DOCTYPE html>
<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=http://s1.bdstatic.com/r/www/cache/bdorz/baidu.min.css><title>百度一下，你就知道</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt value maxlength=255 autocomplete=off autofocus></span><span class="bg s_btn_wr"><input type=submit id=su val

In [5]:
import requests
content=input('请输入你要检索的内容：')
url =f"https://www.sogou.com/web?query={content}"

请输入你要检索的内容：猪


打开开发者工具（F12），打开搜索页面，在network板块下任一XHR的header里观察请求类型

在header里，user-agent中有用户浏览器的信息，可用来处理反爬

In [6]:
headers={
   "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36" 
}
resp=requests.get(url,headers=headers)
print(resp.text)

<!DOCTYPE html><html data-vtype="default"> <head><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta http-equiv="x-dns-prefetch-control" content="on"><meta charset="utf-8"><meta name="referrer" content="always"><meta name="viewport" content="width=device-width,minimum-scale=1,maximum-scale=1,user-scalable=no"><meta name="format-detection" content="telephone=no">  <link rel="dns-prefetch" href="//img01.sogoucdn.com"><link rel="dns-prefetch" href="//img02.sogoucdn.com"><link rel="dns-prefetch" href="//img03.sogoucdn.com"><link rel="dns-prefetch" href="//img04.sogoucdn.com"><link rel="dns-prefetch" href="//dlweb.sogoucdn.com"><link rel="preconnect" href="//img01.sogoucdn.com"><link rel="preconnect" href="//img02.sogoucdn.com"><link rel="preconnect" href="//img03.sogoucdn.com"><link rel="preconnect" href="//img04.sogoucdn.com"><link rel="preconnect" href="//dlweb.sogoucdn.com">  <script type="text/javascript">/*file=static/js/resourceErrorReport.js*/!function(a){var n=(new Date).get

## 处理post请求  
首先post请求很少在url里加入参数，而是在form data里（request header下面）

In [1]:
#让jupyter实现多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # 默认 last_expr

In [4]:
import requests

url="https://fanyi.baidu.com/sug"
mydata={
    "kw":input('请输入一个单词')
}
resp=requests.post(url,data=mydata)

print(resp.text)
#返回的是文本（字符串）
print(resp.json())
#返回的是json语句, .json返回的是字典，可以一层一层往外拿数据

请输入一个单词princess
{"errno":0,"data":[{"k":"princess","v":"n. \u516c\u4e3b; \u738b\u5983\uff08\u738b\u65cf\u5973\u6027\u6210\u5458\uff09; \u5973\u5de8\u5934\uff0c\u5973\u540d\u5bb6; <\u53e4>\u5973\u738b"},{"k":"princesse","v":"adj. \u516c\u4e3b\u88c5\u7684\uff0c\u7d27\u8eab\u8fde\u8863\u88d9\u7684"},{"k":"princesses","v":"n. \u738b\u5983; \uff08\u9664\u5973\u738b\u6216\u738b\u540e\u5916\u7684\uff09\u738b\u5ba4\u5973\u6210\u5458( princess\u7684\u540d\u8bcd\u590d\u6570 ); \uff08\u5c24\u6307\uff09\u516c\u4e3b; \u738b"}]}
{'errno': 0, 'data': [{'k': 'princess', 'v': 'n. 公主; 王妃（王族女性成员）; 女巨头，女名家; <古>女王'}, {'k': 'princesse', 'v': 'adj. 公主装的，紧身连衣裙的'}, {'k': 'princesses', 'v': 'n. 王妃; （除女王或王后外的）王室女成员( princess的名词复数 ); （尤指）公主; 王'}]}


## 处理多参数的get请求  
先在xhr里找到包含参数的文件，在和header里查看相关信息。  
get：query string parameter  
post：form data  



**如何找到包含参数的文件**  
在network里找XHR里的文件，通过preview板块观察是否包含所要请求的数据，如果该文件是所要请求的文件，则通过headers观察其爬虫需要的信息（url参数，request method）  
此处（豆瓣电影）的url参数很长，不方便调参（以问号为分割线）  
get请求的参数在url里

In [8]:
url="https://movie.douban.com/j/chart/top_list?"

url后半部分的参数在payload板块里query string parameter，  
type: 13  
interval_id: 100:90  
action:   
start: 0  
limit: 20  

In [9]:
data={
    "type": "13",
    "interval_id": "100:90",
    "action":"",
    "start": "0",
    "limit": "20",
}

为应对反爬问题，仍旧要对headers作改变

In [10]:
headers={
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36"
}

In [11]:
resp=requests.get(url,params=data,headers=headers)
#注意观察post请求下的语法是requests.post(url,data=mydata)

In [12]:
print(resp.text)

[{"rating":["9.6","50"],"rank":1,"cover_url":"https://img3.doubanio.com\/view\/photo\/s_ratio_poster\/public\/p2561716440.jpg","is_playable":true,"id":"1291546","types":["剧情","爱情","同性"],"regions":["中国大陆","中国香港"],"title":"霸王别姬","url":"https:\/\/movie.douban.com\/subject\/1291546\/","release_date":"1993-07-26","actor_count":26,"vote_count":1884813,"score":"9.6","actors":["张国荣","张丰毅","巩俐","葛优","英达","蒋雯丽","吴大维","吕齐","雷汉","尹治","马明威","费振翔","智一桐","李春","赵海龙","李丹","童弟","沈慧芬","黄斐","徐杰","黄磊","冯远征","杨立新","方征","周璞","隋永清"],"is_watched":false},{"rating":["9.6","50"],"rank":2,"cover_url":"https://img2.doubanio.com\/view\/photo\/s_ratio_poster\/public\/p2578474613.jpg","is_playable":true,"id":"1292063","types":["剧情","喜剧","爱情","战争"],"regions":["意大利"],"title":"美丽人生","url":"https:\/\/movie.douban.com\/subject\/1292063\/","release_date":"2020-01-03","actor_count":29,"vote_count":1168827,"score":"9.6","actors":["罗伯托·贝尼尼","尼可莱塔·布拉斯基","乔治·坎塔里尼","朱斯蒂诺·杜拉诺","赛尔乔·比尼·布斯特里克","玛丽萨·帕雷德斯","霍斯特·布赫霍尔茨","利迪娅·阿方西","朱利亚娜·

In [13]:
print(resp.json())

[{'rating': ['9.6', '50'], 'rank': 1, 'cover_url': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2561716440.jpg', 'is_playable': True, 'id': '1291546', 'types': ['剧情', '爱情', '同性'], 'regions': ['中国大陆', '中国香港'], 'title': '霸王别姬', 'url': 'https://movie.douban.com/subject/1291546/', 'release_date': '1993-07-26', 'actor_count': 26, 'vote_count': 1884813, 'score': '9.6', 'actors': ['张国荣', '张丰毅', '巩俐', '葛优', '英达', '蒋雯丽', '吴大维', '吕齐', '雷汉', '尹治', '马明威', '费振翔', '智一桐', '李春', '赵海龙', '李丹', '童弟', '沈慧芬', '黄斐', '徐杰', '黄磊', '冯远征', '杨立新', '方征', '周璞', '隋永清'], 'is_watched': False}, {'rating': ['9.6', '50'], 'rank': 2, 'cover_url': 'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p2578474613.jpg', 'is_playable': True, 'id': '1292063', 'types': ['剧情', '喜剧', '爱情', '战争'], 'regions': ['意大利'], 'title': '美丽人生', 'url': 'https://movie.douban.com/subject/1292063/', 'release_date': '2020-01-03', 'actor_count': 29, 'vote_count': 1168827, 'score': '9.6', 'actors': ['罗伯托·贝尼尼', '尼可莱塔·布拉斯基', '乔治

In [14]:
print(resp.request.url)

https://movie.douban.com/j/chart/top_list?type=13&interval_id=100%3A90&action=&start=0&limit=20


# 数据解析概述  
1.re解析：正则表达式  
2.bs4解析：代码简洁明了，但效率不高  
3.xpath解析：语法简洁干练，定位好用（重点，执行速度快，现在比较prevalent）  
4.pyquery解析：模仿jqueary前端结构，也比较简单  
四种方式可以混合使用

## regular expression  
正则表达式，一种使用表达式的方式对字符串进行匹配的语法规则  
网页源代码本质上是一个超长的字符串，想从里面提取内容可以用正则表达式  
  
正则表达式的有点：速度快，效率高，准确性高（语法严谨）  
正则的缺点：难度稍高  
  
只要掌握了正则编写的逻辑关系，写出一个提取页面内容的正则其实并不复杂  


正则表达式在线测试网页：[https://tool.oschina.net/regex](https://tool.oschina.net/regex)  
原始字符串  
正则表达式  
匹配结果

正则是按位置匹配的  
类型：普通字符、元字符（具有固定含义的特殊符号，可以匹配一类字符）

### 元字符：  
**.**：匹配除了换行外的任意字符，未来在Python的re模块中是一个坑（）  
**\w**:任意字母、数字和下划线  
\s:匹配任意空白符（用不着）  
**\d**:匹配数字（一个元字符匹配一位，五个匹配五位）  
\n:换行符  
\t:制表符  
^x:以x开头，x可以是字符也可以使元字符  
x\\$:以x结尾，也可以与^结合使用，如^x\\$  
  
大写表示非  
\W:除字母数字下划线的字符  
\D:除数字外  
\S:除空白符外  
|：或者，可以用括号引（xxx）|（yyy）  
[a-z]:表示匹配所有小写字母  
[a-zA-Z0-9]：匹配所有字母+数字  
[^a-z]:除了字符组中的所有字符

### 量词  
x* 重复零次或更多次  
x+ 重复一次或更多次  
x? 重复零次或一次  
{n} 重复n次  
{n,} 重复n次或更多次  
{n,m} 重复n次到m次

### 贪婪和惰性匹配   
.* ：（主要是\*）贪婪匹配，尽可能多地得到结果  
x.\*y:从x到y最长的字符  
  
.\*?:惰性匹配，尽可能少地匹配  
x.\*?y:从x到y最少的字符（回溯）

常用例子：`<div>.\*?</div>`  
后面可以将内容单拎出来

### re模块  
Python内置模块，功能不多，主要用于匹配字符串

#### findall：拿到所有匹配结果

In [15]:
import re
result=re.findall("a","a救命a")
print(result)

['a', 'a']


In [16]:
result=re.findall(r"\d+","a救命1981987aloveyou3000times")
print(result)

['1981987', '3000']


#### finditer：迭代

In [20]:
result=re.finditer(r"\d+","123a救命aloveyou3000times")
print(result)

In [21]:
##重点
result=re.finditer(r"\d+","a123救19871987命aloveyou3000times")
for item in result:#从迭代器里拿内容
    print(item.group())#从匹配结果中拿数据

123
19871987
3000


#### search：只拿到第一项匹配结果

In [23]:
result=re.search(r"\d+","123a救命aloveyou3000times")
print(result.group())

123


#### match:类似于^正则   
从字符串的开头进行匹配

In [29]:
result=re.match(r"\d+","a我123救命aloveyou3000times")
print(result)

None


#### 预加载  
循环里反复使用同一正则语句势必要加载内存，可以利用预加载提前把正则对象加载完毕

In [37]:
obj= re.compile(r"\d+")
for item in "a我123救命aloveyou3000times":
    result=obj.finditer(item)
print(result)

In [26]:
obj= re.compile(r"\d+")
result=obj.finditer("a我123救命aloveyou3000times")
for item in result:
    print(item.group())

123
3000


In [39]:
obj= re.compile(r"\d+")
result=obj.findall("a我123救命aloveyou3000times")
print(result)

['123', '3000']


#### re提取分组数据  
假设现有字符串`<div class='西游记'><span id='10010'>中国联通</span></div>`

In [24]:
s="""
<div class='西游记'><span id='10010'>中国联通</span></div>
<div class='西游记'><span id='10010'>中国联通</span></div>`
"""

In [25]:
obj=re.compile(r"<span id='(\d+)'>(.*?)</span>")#用小括号把需要提取的内容括起来
result=obj.findall(s)
print(result)

[('10010', '中国联通'), ('10010', '中国联通')]


In [32]:
obj=re.compile(r"<span id='(?P<id>\d+)'>(?P<name>.*?)</span>")#用?P<id>表明提取数据的变量名，再通过group拿到迭代器（finditer里）
result=obj.finditer(s)
for item in result:
    id=item.group("id")
    print(id)
    name=item.group("name")
    print(name)

10010
中国联通
10010
中国联通


### 正则实例：豆瓣电影top250  
第一步是先看页面源代码里面有没有需要的数据。豆瓣有。  
思路：  
1.拿到页面源代码  
2.编写正则提取页面数据  
3.保存数据

In [27]:
import requests
import re

In [28]:
f=open("top250.csv",mode="w",encoding='utf-8')#记得在循环外打开文件，否则就是在一遍遍重写文件，最后只能保留最后一页的数据
for i in range(0,250,25):
    url = "https://movie.douban.com/top250?start="+str(i)
    headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36"
    }
    resp=requests.get(url,headers=headers)
    content=resp.text

    #re.S可以让正则中的.匹配换行符
    obj=re.compile(r'<div class="item">.*?<span class="title">(?P<name>.*?)</span>.*?<p class="">.*?导演: (?P<director>.*?)&nbsp.*?<br>(?P<year>.*?)&nbsp.*?<span class="rating_num" property="v:average">(?P<score>.*?)</span>.*?<span>(?P<num>.*?)人评价</span>',re.S)
    result=obj.finditer(content)


    for item in result:
        name=item.group("name")
        director=item.group("director")
        year=item.group("year").strip()#去掉年份里（字符串左右两端）的空白符
        score=item.group("score")
        num=item.group("num")
        f.write(f"{name},{director},{year},{score},{num}\n")

f.close()
resp.close()
print("大功告成啦")

大功告成啦


### 正则实例：电影天堂  
思路：  
1.提取主页面里每个电影背后的url  
2.访问子页面，提取到电影名称以及下载地址  


在HTML文件里，`<a></a>`引的是页面中要显示的文本  
`<a href="url">text</a>`中href后面跟的是链接  
<a href="http://www.baidu.com">腾讯巴巴</a>

In [29]:
import requests
import re

In [22]:
url="https://www.dy2018.com/"
resp=requests.get(url)
resp.encoding="gbk"
print(resp.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no">
    <META http-equiv=Content-Type content="text/html; charset=gb2312">
    <title>电影天堂_电影下载_高清首发</title>
    <META content="电影天堂,迅雷电影,电影下载,迅雷电影下载" name=keywords>
    <META content="最好的迅雷电影下载网，分享最新电影，高清电影、综艺、动漫、电视剧等下载！" name=description>
<script type="text/javascript" src="/js/head.js"></script>
    <link href="/css/dygod.css?=1023" rel="stylesheet" type="text/css" />
    <script language=javascript>kstatus(); function kstatus(){self.status="喜欢本站请分享给您的朋友哦，TA会感谢您，谢谢支持！"; setTimeout("kstatus()",0);}</script>
</head><body style="width:960px;margin:0 auto;">
<div id="header"><div class="contain"><h4><a href="/"></a></h4>
    <div id="headerright"><div id="about" align="right">
        <SCRIPT src="/jsdd/760h.js"></

In [23]:
#提取所要求内容的HTML代码
obj1=re.compile(r"2021必看热片.*?<ul>(?P<html>.*?)</ul>",re.S)
result1=obj1.search(resp.text)
html=result1.group("html")
print(html)


                        <li><a href='/i/104291.html' title="2021年中国香港7.5分动作犯罪片《怒火·重案》BD国粤双语中字">2021年中国香港7.5分动作犯罪片《怒火·重案》BD国粤双语中字</a><span><font color=#FF0000>11-21</font></span></li>
<li><a href='/i/104554.html' title="2021年美国6.6分喜剧犯罪片《红色通缉令》BD中英双字">2021年美国6.6分喜剧犯罪片《红色通缉令》BD中英双字</a><span><font color=#FF0000>11-13</font></span></li>
<li><a href='/i/104394.html' title="2021年韩国8.2分惊悚历史片《摩加迪沙》BD韩语中字">2021年韩国8.2分惊悚历史片《摩加迪沙》BD韩语中字</a><span><font color=#FF0000>11-08</font></span></li>
<li><a href='/i/104516.html' title="2021年欧美8.3分科幻片《芬奇》BD英语中字">2021年欧美8.3分科幻片《芬奇》BD英语中字</a><span><font color=#FF0000>11-07</font></span></li>
<li><a href='/i/102248.html' title="2019年日本7.4分科幻动画片《你好世界》BD国日语双语中字">2019年日本7.4分科幻动画片《你好世界》BD国日语双语中字</a><span><font color=#FF0000>11-02</font></span></li>
<li><a href='/i/104492.html' title="2021年国产6.3分动作灾难片《峰爆》HD国语中字">2021年国产6.3分动作灾难片《峰爆》HD国语中字</a><span><font color=#FF0000>10-30</font></span></li>
<li><a href='/i/104490.html' title="2021年欧美6.9分动作犯罪片《神偷军团》BD中字">2021年欧美6.9分

In [25]:
#提取a标签中的href值
obj2=re.compile(r"<li><a href='/(?P<href>.*?)' title=",re.S)
obj3=re.compile(r'<div id="Zoom">.*?◎片　　名　(?P<name>.*?)<br />.*?<td style="WORD-WRAP: break-word" bgcolor="#fdfddf"><a href="(?P<download>.*?)">',re.S)

result2=obj2.finditer(html)
for item in result2:
    #print(item.group("href"))
    child_url=url+item.group("href")
    child_resp=requests.get(child_url)
    child_resp.encoding="gbk"
    #print(child_resp.text)
    
    result3=obj3.search(child_resp.text)
    name=result3.group("name")
    download=result3.group("download")
    print(name,download)

怒火
magnet:?xt=urn:btih:45b38416cabbeaa5320556c297d4a4d92eb775cd&dn=[电影天堂www.dytt89.com]怒火·重案-2021_BD国粤双语中字.mp4&tr=http://t.t789.me:2710/announce&tr=http://t.t789.co:2710/announce&tr=http://t.t789.vip:2710/announce
Red Notice
magnet:?xt=urn:btih:09b6a2670b7c71708fa202b9c3524b45ed56a6af&dn=[电影天堂www.dytt89.com]红色通缉令-2021_BD中英双字.mp4&tr=http://t.t789.me:2710/announce&tr=http://t.t789.co:2710/announce&tr=http://t.t789.vip:2710/announce
&#47784;&#44032;&#46356;&#49800;
magnet:?xt=urn:btih:1e859bb3a66680b1c4416b1d51b4db09276bb46f&dn=[电影天堂www.dytt89.com]摩加迪沙-2021_BD韩语中字V2.mp4&tr=http://t.t789.me:2710/announce&tr=http://t.t789.co:2710/announce&tr=http://t.t789.vip:2710/announce
Finch
magnet:?xt=urn:btih:abaa46ecd9b262406808056d0039ce672cdf6a44&dn=[电影天堂www.dytt89.com]芬奇-2021_BD英语中字.mp4&tr=http://t.t789.me:2710/announce&tr=http://t.t789.co:2710/announce&tr=http://t.t789.vip:2710/announce
HELLO WORLD
magnet:?xt=urn:btih:402989db7a00bc99e0622e7f4c18934d8fa8b539&dn=[电影天堂www.dytt89.com]你好世界-2019_BD国日语

# 前端基础语法  
## HTML基础语法  
HTML（hyper text markup language）：超文本标记语言，用不同标签对网页上的内容进行标记`<></>`  
`<h1 align="right">这是网页中要展示的标题</h1>`<标签名 属性="值">被标记的内容</标签名>  
`<img src="abc.jpg" width="50px" height="60px"/>`<标签名 属性="值" 被标记的内容/>.再比如`</br>`表示换行

In [26]:
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Title</title>
    </head>
    <body bgcolor="pink">
        <h1 align="right">这是网页中要展示的标题</h1>
        <p>这是某一个段落</p>
        <a href="url">前一个标签里多余的那部分是属性=value，补充标签的展示效果</a>
    <img src="abc.jpg"/>
    </body>
</html>

SyntaxError: invalid syntax (<ipython-input-26-d499085d5229>, line 1)

## CSS基础语法  
CSS（cascading style sheets）：层叠样式表，用来定义页面内容展示效果。  
HTML是骨架，CSS是页面效果美化，主要是学CSS的语法规则和选择器（重点） 

### CSS语法规则
` <div style="width:100px;height:100px;border:1px solid red;">hello world</div>`style属性添加css  
` <style>
    span{
        color:pink;
    }
</style>`在head里前置style标签添加css  
`<link rel="stylesheet" href="mycss.css">`另外写一个css文件，里面可能包括对样式的规定

In [ ]:
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Title</title>
        <link rel="stylesheet" href="mycss.css">
        <style>
            span{
                color:pink;
            }
        </style>
    </head>
    <body bgcolor="pink">
        <div style="width:100px;height:100px;border:1px solid red;">hello world</div>
        <span></span>
    </body>
</html>

### CSS选择器  
前三种是常用的

#### id选择器   
`#abc{
    color:pink;
}`

#### 标签选择器 标签  
`li{
    border:1px solid red;
}`  

#### 类选择器  .  
`.meimei{
    color:green;
}`  

In [ ]:
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Title</title>
        <style>
            \#abc{
                color:pink;
            }
            li{
                border:1px solid red;
            }
            .meimei{
                color:green;
            }
        </style>
    </head>
    <body bgcolor="pink">
        <ul>
            <li id="abc">elsa</li>
            <li>anna</li>
            <li>coco</li>
            <li>jojo</li>
        </ul>
        
        <div class="meimei">这是个div</div>
        <span class="meimei">这是个span</span>
    </body>
</html>

#### 选择器分组 ,  
`div,span{
    background:hotpink;
}`

#### 后代选择权 空格  
`ul li{
    list-style:none;
}`  
在ul里的所有li标签（包括子标签里的li，也包括子子孙孙balabala）都会适用这种格式

#### 子选择器 >  
`ol>li{
    background:green;
}`  
只有直系一层的子标签里的li标签适用这种格式

In [ ]:
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Title</title>
        <style>
            div,span{
                background:hotpink;
            }
            ul li{
                list-style:none;
            }
            ol>li{
                background:green;
            }
        </style>
    </head>
    <body bgcolor="pink">
        <ul>
            <li id="abc">elsa</li>
            <li>anna</li>
            <li>coco</li>
            <li>jojo</li>
        </ul>
        <ol>
            <li>yoyo</li>
            <li>check it out</li>
            <li>here we go</li>
        </ol>
        
        <div class="meimei">这是个div</div>
        <span class="meimei">这是个span</span>
    </body>
</html>

#### 相邻选择器 +  
`h1+h2{
    color:red;
}`  
只有跟h1相邻的h2才会适用

#### 属性选择器 \[属性=值]  
`a[target]{
    color:green;
}
a[target='-blank']{
    text-decoration:none;
}`

In [27]:
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Title</title>
        <style>
            h1+h2{
                color:red;
            }
            a[target]{
                color:green;
            }
            a[target='-blank']{
                text-decoration:none;
            }
        </style>
    </head>
    <body bgcolor="pink">
        <h1>这是个h1</h1>
        <h2>这是个h2</h2>
        <h2>这才不是h2</h2>
        <a href="#" target="_blank">呜呜呜</a>
        <a href="#" target="_self">嘤嘤嘤</a>
        <a href="#">啾啾啾</a>
    </body>
</html>

SyntaxError: invalid syntax (<ipython-input-27-1cab80aa3f9e>, line 1)

# bs4

## bs4基本使用  
先安装

In [29]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=b7160941823235c736f27f1eccd24d5d776382df409fdb3bee86a2c232c06fb8
  Stored in directory: c:\users\18810\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [30]:
from bs4 import BeautifulSoup
import re
import requests

In [45]:
html="""
<ul class=" undefined list__09f24__ynIEd">
    <li class=" border-color--default__09f24__NPAKY"><a href="zhangwuji.com">张无忌</a></li>
    <li id="abc"><a href="zhouxingchi.com">周星驰</a></li>
    <li class=" border-color--default__09f24__NPAKY"><a href="zhubajie.com">猪八戒</a></li>
    <li class=" border-color--default__09f24__NPAKY"><a href="wuzetian.com">武则天</a></li>
</ul>
"""

In [31]:
page=BeautifulSoup(html,"html.parser")#初始化对象，用html的解析系进行解析
page.find("tagname",attrs={"属性":"值"})#查找某个元素，后可接option，find只找一个就返回
page.find_all("tagname",attrs={"属性":"值"})#找到一堆结果并返回

In [46]:
page=BeautifulSoup(html,"html.parser")#用html的解析系进行解析

In [42]:
abc=page.find("li",attrs={"id":"abc"})#find只找一个就返回
print(abc)

<li id="abc"><a href="zhouxingchi.com">周星驰</a></li>


In [41]:
a=abc.find("a")
#想要啥就去找哪个属性
print(a)
print(a.text)
print(a.get("href"))

<a href="zhouxingchi.com">周星驰</a>
周星驰
zhouxingchi.com


In [47]:
li_list=page.find_all("li")
for li in li_list:
    a=li.find("a")
    text=a.text
    href=a.get("href")
    print(text,href)

张无忌 zhangwuji.com
周星驰 zhouxingchi.com
猪八戒 zhubajie.com
武则天 wuzetian.com


## bs4实例  
HTML里的表格以`<table>`和`</table>`为开头和结尾，`<tr>`和`</tr>`表示一行，`<td>`和`</td>`为一个单元格  
这个例子网页失效了，主要看代码怎么写吧。

In [ ]:
from bs4 import BeautifulSoup
import re
import requests

In [ ]:
f=open("xxx.csv",mode="w",encoding="utf-8")

url=""
resp=requests.get(url)
page=BeautifulSoup(resp.text,'html.parser')
table=page.find("table",attrs={"class": "hours-table"})
trs=table.findall("tr")[1:]#意思是拿第一行外的所有tr
for tr in trs:
    tds=tr.find_all("td")
    name=tds[0].text#要的是文本
    low=tds[1].text
    avg=tds[2].text
    high=tds[3].text
    scale=tds[4].text
    unit=tds[5].text
    date=tds[6].text
    f.write(f"{name},{low},{avg}\n")
f.close()
resp.close()
print("好了吗")

## bs实例2  
图片信息，不尝试了，主要看代码

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url=domain="https:www.umei.net/bizhitupian/xiaoqingxinbizhi"

resp=requests.get(url)
resp.encoding="utf-8"

main_page=BeautifulSoup(resp.text,"html.parser")
a_list=main_page.find_all("a",attrs={"class":"TypeBigPics"})
print(len(a_list))#检查拿到了正确数量的a标签
"""
子页面url的开头若为/，直接在前面拼接上域名即可
子页面url开头若不为/，此时需要找到主页面的url，去掉最后一个/后面的所有内容，和当前获取的url进行拼接
"""
domain="https:www.umei.net/bizhitupian/xiaoqingxinbizhi"

n=1
for a in a_list:
    href=a.get("href")
    child_url=domian+href
    child_resp=requests.get(child_url)
    #print(child_resp.text)
    child_resp.encoding="utf-8"
    #子页面的bs对象
    child_bs=BeautifulSoup(child_resp.text,"html.parser")
    div=child_bs.find("div",attrs={"class":"ImageBody"})
    img_src=div.find("img").get("src")
    #print(img_src)
    img_resp=requests.get(img_src)#这不是文本，没有text属性
    with open(f"{n}.jpg",mode="wb")as f:
        f.write(img_resp.content)#把图片的字节写入文件中，因为是字节所以必须是wb的mode
    print(f"{n}下载完毕")
    n=n+1


# xpath  
xpath是一门在xml文档中查找信息的语言，xpath可以在xml文档中对元素和属性进行遍历。HTML是xml的一个子集。

## xpath基本语法

In [ ]:
#xml范例，<不叫标签叫节点>：父子标签→父子节点
<book>
    <id>l</id>
    <name>flower</flower>
    <price>1.23</price>
    <author>
        <nick>周大强</nick>
        <nick>周芷若</nick>
    </author>
</book>

In [1]:
from lxml import etree

In [4]:
xml="""
<book>
    <id>l</id>
    <name>flower</name>
    <nick>momo</nick>
    <price>1.23</price>
        <nick>yoyo</nick>
    <author>
        <nick>周大强</nick>
        <nick>周芷若</nick>
        <nick class="jay">周jay</nick>
    </author>
    <partner>
        <nick id="miaomiao">累死</nick>
        <nick id="miumiu">人了</nick>
    </partner>
</book>
"""
#etree.HTML
et=etree.XML(xml)
result0=et.xpath("/book")#/表示根节点
result=et.xpath("/book/name/text()")[0]#在xpath中间的/表示子节点，并且xpath默认返回列表
result=et.xpath("/book//nick")#//表示所有子孙节点
result=et.xpath("/book/*/nick/text()")#这样表示任意子节点的子节点，即必须是孙节点才会被返回,*表示通配符
result=et.xpath("/book/author/nick[@class='jay']/text()")#nick节点的@class='jay'表示class属性为jay的nick节点
result=et.xpath("/book/partner/nick/@id")#最后一个/表示拿到你nick里的id属性


In [ ]:
html="""
"""
et=etree.HTML(html)
li_list=et.xpath("/html/body/ul/li")#li的列表
li_wanted=et.xpath("/html/body/ul/li[2]")

li_list=et.xpath("//li")#定位到页面里所有li
for li in li_list:
    href=li.xpath("./a/@href")[0]#./表示当前节点，当前标签就是li
    text=li.path("./a/text()")[0]
    print(text,href)#成对数据，方便后续爬虫


## xpath实例  
思路：  
1.页面源代码  
2.从页面源代码中提取所需数据

In [2]:
import requests
from lxml import etree

In [3]:
url="https://beijing.zbj.com/search/f/?type=new&kw=saas"
resp=requests.get(url)
resp.encoding="utf-8"
print(resp.text)

<!DOCTYPE html>
<html>
<head>
<meta charset='utf-8'/>
<meta http-equiv='X-UA-Compatible' content='IE=edge,chrome=1'/>
<meta name='renderer' content='webkit'/>
<meta http-equiv='Cache-Control' content='no-transform '/>




<title>【北京saas价格_北京saas报价】_北京saas服务外包信息-北京猪八戒网</title>
<meta name='description' content='北京saas服务搜索页，从价格、评价、销量出发，为您精选了34903个北京saas相关的服务外包报价信息，为您购买北京saas服务，提供有价值的参考，查找更多北京saas相关的服务外包信息，尽在猪八戒北京saas网。&#39;'/>
<meta name='keywords' content='北京saas报价，北京saas价格，北京saas服务外包信息，北京saas价格信息'/>
<script>
utopia.arrived({uid:'utopia_widget_1', js:[], entry:function(){}});
</script>

<link rel='shortcut icon' href='//s.zbjimg.com/img/favicon.ico' type='image/x-icon'/>
<script type='text/javascript'>/*<![CDATA[*/

window.ZBJInfo = {
baseURI: "zbj.com", 
pageDomain: "zbj.com",
staticLibURI: "//as.zbjimg.com", 
asStaticLibURI: "//as.zbjimg.com", 
qiniuUploadTokenUrl: "//rms.zhubajie.com",
runtime: "product",
kw: "saas",
categoryId: null,
categoryNameReal: "",
pageType: "services",
pageTy

In [22]:
et=etree.HTML(resp.text)
divs=et.xpath("//div[@class='new-service-wrap']/div")
for div in divs:
    #div是一条商品的信息
    #注意elements里是脚本运行后的结果，只能作为辅助
    price=div.xpath("./div/div/a/div[2]/div[1]/span[1]/text()")
    if not price:#过滤无用数据
        continue
    company_raw=div.xpath("./div/div/a[1]/div[1]/p/text()")[1]
    company=company_raw.strip("\n")
    name=div.xpath("./div/div/a[2]/div[2]/div[2]/p//text()")#//表示该标签下所有子孙标签的text
    #print(name)#这里返回的是一些列表（可能2-3列）
    #r="_".join(["张无忌","马化腾"])表示用"_"连接两个字符串
    name="".join(name)
    print(name)



APP定制开发设计saas电商城教育直播iOS应用成品工具d
saas支付宝微信公众号小程序商城H5python开发定制作
微电商|微信开发|SaaS|微信|微信公众号|微信商城|分销
软件UI界面web管理系统saas平台后台客户端终端页面设计
其它工具软件开发系统软件其它类微信H5游戏SAAS软件CRM
app小程序开发APP网站建设软件saas后端python
北京软件开发|OA SAAS ERP CRM 功能开发|系统
软件开发|系统定制|OA SAAS ERP CRM 功能开发
仓储物流软件出入库数据库IT开发定制SaaS企业管理系统平台
OA进销存Crm项目管理Erp系统saas软件开发SAP定制
北京软件开发系统开发大型连锁超市SaaS软件管理系统
软件UI界面web管理系统saas平台后台客户端终端页面设计
北京软件开发|OA SAAS ERP CRM 功能开发|系统
软件开发|系统定制|OA SAAS ERP CRM 功能开发
CRM|OA|saas会员管理系统|进库存系统|定制餐饮开发
saas支付宝微信公众号小程序商城H5python开发定制作
软件开发/APP定制开发/ERP开发/HR开发/SAAS系统
微信小程序开发saas定制设计php公众号H5点餐交友算法c
软件开发企业管理软件开发SaaS工具软件考勤财务系统定制开发
外卖支付宝移动应用UI设计可视化小程序软件数据库saas设计
软件界面UI网页OA办公智能监测统计系统后台管理设计SaaS
java服务工具python数据分析处理saas系统软件开发
北京软件开发|OA SAAS ERP CRM 功能开发|系统
SaaS系统|CRM系统|微信开发|微信公众平台开发|分销
小程序开发灵活用工系统开发 系统搭建私域服务器 SAAS平台
管理软件界面区块链ui进销存saas界面ERP设计办公ui
微信评选系统开发saas平台公众号评选系统活动营销场景搭建
开发工具企业管理行业应用嵌入式区块链应用SaaS成品设计网站
企业微信开发-办公小程序教育家校通企微管家SaaS应用政务
系统界面软件ui后台管理网页设计saas设计erp设计网站
微信小程序商城带分销系统成品SAAS软件 包安装支持二次开发
【SAAS定制开发】C/S定制开发|软件服务|智能平台
微信小程序开发saas定制设计php公众号

In [ ]:
from PyPDF2 import PdfFileWriter, PdfFileReader

wd = 'D:/Documents/graduate/david yang-ra/land_value_data/'

In [ ]:
infile = "1933E.pdf"
outfile = "1933E.pdf"
output = PdfFileWriter()
input_pdf = PdfFileReader(open(infile, wd))
output_file = open(outfile, wd)
page_ranges = (x.split("-") for x in page_range.split(","))
range_list = [i for r in page_ranges for i in range(int(r[0]), int(r[-1]) + 1)]
for p in range_list:
    output.addPage(input_pdf.getPage(p - 1))
output.write(output_file)